<a href="https://colab.research.google.com/github/felipeabe/artificial-neural-network/blob/feature%2Fentrega-final/notebooks/trabalhoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Remover o diretório 'artificial-neural-network' se já existir
!rm -rf /content/artificial-neural-network

# Clonar o repositório do GitHub no ambiente do Colab
!git clone --branch feature/entrega-final https://github.com/felipeabe/artificial-neural-network.git

# Adicionar o diretório src ao sys.path para permitir a importação de módulos
import sys
sys.path.append('/content/artificial-neural-network/src')

# Verificar se o repositório foi clonado corretamente
!ls /content/artificial-neural-network



Cloning into 'artificial-neural-network'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 130 (delta 58), reused 37 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (130/130), 909.92 KiB | 7.16 MiB/s, done.
Resolving deltas: 100% (58/58), done.
datasets  notebooks  README.md	src  teste.ipynb


In [5]:
# Importando bibliotecas necessárias
import pandas as pd

# Carregar o dataset 'obesity.csv'
# Este conjunto contém dados sobre obesidade com fatores como estilo de vida e transporte
dados_obesidade = pd.read_csv("/content/artificial-neural-network/datasets/obesity.csv")

# Renomear colunas para nomes mais intuitivos
mapear_colunas = {
    'family_history_with_overweight': 'historico_familiar',
    'FAVC': 'comida_calorica',
    'FCVC': 'vegetais',
    'NCP': 'refeicoes_dia',
    'SMOKE': 'fuma',
    'CH2O': 'agua_dia',
    'SCC': 'calorias',
    'FAF': 'atividade_fisica',
    'CALC': 'alcool',
    'MTRANS': 'transporte',
    'NObeyesdad': 'nivel_obesidade'
}
dados_obesidade = dados_obesidade.rename(columns=mapear_colunas)

# Remover colunas irrelevantes para a análise
colunas_remover = ['CAEC', 'TUE']
dados_obesidade = dados_obesidade.drop(columns=colunas_remover)

# Codificar variáveis categóricas (binárias e escalares)
mapeamento_binario = {
    'Gender': {"Female": 0, "Male": 1},
    'historico_familiar': {"yes": 1, "no": 0},
    'comida_calorica': {"yes": 1, "no": 0},
    'fuma': {"yes": 1, "no": 0},
    'calorias': {"yes": 1, "no": 0}
}
for coluna, mapeamento in mapeamento_binario.items():
    dados_obesidade[coluna] = dados_obesidade[coluna].map(mapeamento)

# Codificar consumo de álcool e transporte
mapeamento_alcool = {'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3}
dados_obesidade['alcool'] = dados_obesidade['alcool'].map(mapeamento_alcool)

mapeamento_transporte = {
    'Walking': 0, 'Bike': 0,
    'Motorbike': 1, 'Automobile': 1,
    'Public_Transportation': 1
}
dados_obesidade['transporte'] = dados_obesidade['transporte'].map(mapeamento_transporte)

# Normalizar dados contínuos (altura, peso, idade)
colunas_normalizar = ['Height', 'Weight', 'Age']
dados_obesidade[colunas_normalizar] = dados_obesidade[colunas_normalizar].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

# Dividir o dataset em treino (80%) e teste (20%)
from sklearn.model_selection import train_test_split
X_obesidade = dados_obesidade.drop(columns=["nivel_obesidade"])
y_obesidade = dados_obesidade["nivel_obesidade"]
X_treino_multiclasse, X_teste_multiclasse, y_treino_multiclasse, y_teste_multiclasse = train_test_split(
    X_obesidade, y_obesidade, test_size=0.2, random_state=42
)


In [6]:
# Importar bibliotecas necessárias
import pandas as pd

# Carregar o dataset 'houses.csv'
dados_imoveis = pd.read_csv("/content/artificial-neural-network/datasets/houses.csv")

# Remover colunas irrelevantes
colunas_remover = ['renovated', 'quartile_zone']
dados_imoveis = dados_imoveis.drop(columns=colunas_remover)

# Codificar valores booleanos (True/False) em numéricos (1/0)
dados_imoveis = dados_imoveis.replace({True: 1, False: 0})

# Extrair o ano da coluna de data
dados_imoveis['date'] = pd.to_datetime(dados_imoveis['date'])
dados_imoveis['ano_construcao'] = dados_imoveis['date'].dt.year
dados_imoveis = dados_imoveis.drop(columns=['date'])

# Normalizar colunas contínuas ('price' e 'living_in_m2') para valores entre 0 e 1
colunas_normalizar = ['price', 'living_in_m2']
dados_imoveis[colunas_normalizar] = dados_imoveis[colunas_normalizar].apply(
    lambda x: ((x - x.min()) / (x.max() - x.min())).round(3)
)

# Dividir o conjunto de dados em treino (80%) e teste (20%)
from sklearn.model_selection import train_test_split
X_imoveis = dados_imoveis.drop(columns=["price"])
y_imoveis = dados_imoveis["price"]
X_treino_regressao, X_teste_regressao, y_treino_regressao, y_teste_regressao = train_test_split(
    X_imoveis, y_imoveis, test_size=0.2, random_state=42
)


<ipython-input-6-0ed7743b9e9b>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados_imoveis = dados_imoveis.replace({True: 1, False: 0})


In [7]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Carregar o dataset 'alzheimer.csv'
dados_alzheimer = pd.read_csv("/content/artificial-neural-network/datasets/alzheimer.csv")

# Remover colunas irrelevantes
dados_alzheimer = dados_alzheimer.drop(columns=["PatientID", "DoctorInCharge"])

# Separar variáveis categóricas e numéricas
variaveis_categoricas = ["Gender", "Ethnicity", "EducationLevel"]
variaveis_numericas = [col for col in dados_alzheimer.columns if col not in variaveis_categoricas + ["Diagnosis"]]

# Codificar variáveis categóricas
codificador = OneHotEncoder(drop="first", sparse_output=False)
categorias_codificadas = codificador.fit_transform(dados_alzheimer[variaveis_categoricas])
categorias_codificadas_df = pd.DataFrame(categorias_codificadas, columns=codificador.get_feature_names_out(variaveis_categoricas))

# Normalizar variáveis numéricas
escalador = StandardScaler()
numericas_normalizadas = escalador.fit_transform(dados_alzheimer[variaveis_numericas])
numericas_normalizadas_df = pd.DataFrame(numericas_normalizadas, columns=variaveis_numericas)

# Combinar dados processados
dados_processados = pd.concat([numericas_normalizadas_df, categorias_codificadas_df], axis=1)
dados_processados["Diagnosis"] = dados_alzheimer["Diagnosis"]

# Dividir os dados em conjuntos de treino e teste
X_alzheimer = dados_processados.drop(columns=["Diagnosis"])
y_alzheimer = dados_processados["Diagnosis"]
X_treino_binario, X_teste_binario, y_treino_binario, y_teste_binario = train_test_split(
    X_alzheimer, y_alzheimer, test_size=0.2, random_state=42
)


In [9]:
# Adicionar o diretório `src` ao caminho do Python
# Isso permite que módulos personalizados no diretório src sejam importados diretamente no ambiente Python.
import sys
sys.path.append('/content/artificial-neural-network/src')

# Importar a implementação da Rede Neural
# Importa as classes e funções necessárias para a configuração e treinamento do modelo de Rede Neural Artificial.
from neural_network import RedeNeural
from activations import relu, sigmoid, tanh
from losses import mse_loss, cross_entropy_loss

# Selecionar o conjunto de dados a ser utilizado no treinamento
# Aqui, utilizamos o conjunto de treino e teste processado para a tarefa de classificação binária.
# Esta escolha deve ser ajustada dependendo da tarefa: regressão, classificação binária ou multiclasse.
X_treino = X_treino_binario
y_treino = y_treino_binario
X_teste = X_teste_binario
y_teste = y_teste_binario

# Configuração dos parâmetros da Rede Neural
# Estes parâmetros definem a estrutura do modelo e os hiperparâmetros do processo de treinamento:
# - `input_size`: número de características de entrada no conjunto de treino.
# - `hidden_size`: número de neurônios na camada oculta, influenciando a capacidade do modelo de capturar padrões complexos.
# - `output_size`: número de neurônios na camada de saída, ajustado de acordo com a tarefa (binária ou multiclasse).
# - `activation`: função de ativação usada para introduzir não-linearidades no modelo.
# - `loss`: função de perda para medir o erro durante o treinamento.
# - `learning_rate`: taxa de aprendizado que controla o tamanho dos ajustes nos pesos.
# - `epochs`: número de iterações completas sobre o conjunto de dados de treino.
configuracao = {
    "input_size": X_treino.shape[1],  # Número de características no conjunto de treino
    "hidden_size": 10,  # Número de neurônios na camada oculta
    "output_size": 1,  # Saída com 1 neurônio (para classificação binária)
    "activation": "relu",  # Função de ativação escolhida
    "loss": "mse",  # Função de perda escolhida
    "learning_rate": 0.01,  # Taxa de aprendizado para o gradiente descendente
    "epochs": 1000  # Número de épocas de treinamento
}

# Instanciar a Rede Neural
# A classe `RedeNeural` representa uma rede neural de múltiplas camadas com capacidade de realizar propagação para frente,
# retropropagação e ajuste dos pesos baseados no gradiente descendente.
rede_neural = RedeNeural(
    entrada_tamanho=configuracao["input_size"],  # Número de características de entrada
    oculta_tamanho=configuracao["hidden_size"],  # Número de neurônios na camada oculta
    saida_tamanho=configuracao["output_size"],  # Número de neurônios na camada de saída
    ativacao=configuracao["activation"],  # Função de ativação configurada
    perda=configuracao["loss"]  # Função de perda configurada
)

# Treinar a Rede Neural com os dados de treino
# O método `treinar` realiza o ajuste dos pesos através do algoritmo de retropropagação,
# utilizando a função de ativação e a função de perda configuradas. O treinamento ocorre por várias épocas.
rede_neural.treinar(
    X_treino.to_numpy(),  # Dados de entrada para treinamento (convertidos para numpy)
    y_treino.to_numpy().reshape(-1, 1),  # Rótulos de saída ajustados para o formato adequado
    configuracao["epochs"],  # Número de épocas
    configuracao["learning_rate"]  # Taxa de aprendizado
)

# Avaliar a Rede Neural com os dados de teste
# Após o treinamento, a avaliação do modelo é realizada utilizando o conjunto de teste.
# A acurácia é calculada comparando as predições da rede com os rótulos verdadeiros.
y_predito = rede_neural.propagacao_frente(X_teste.to_numpy())  # Propagação para frente no conjunto de teste
acuracia = (y_predito.round() == y_teste.to_numpy().reshape(-1, 1)).mean() * 100  # Cálculo da acurácia
print(f"Acurácia do modelo: {acuracia:.2f}%")  # Exibição da acurácia do modelo


Época 0/1000, Perda: 0.2501
Época 100/1000, Perda: 0.2415
Época 200/1000, Perda: 0.2363
Época 300/1000, Perda: 0.2332
Época 400/1000, Perda: 0.2312
Época 500/1000, Perda: 0.2300
Época 600/1000, Perda: 0.2292
Época 700/1000, Perda: 0.2285
Época 800/1000, Perda: 0.2279
Época 900/1000, Perda: 0.2272
Acurácia do modelo: 64.42%


Treinamento para Classificação Binária

In [10]:
# Configuração para o treino de classificação binária
# Este bloco implementa o treinamento para tarefas de classificação binária, como prever duas classes possíveis.
# Exemplos: diagnóstico positivo/negativo ou ativado/desativado.

# Selecionar o conjunto de dados para classificação binária
X_treino = X_treino_binario
y_treino = y_treino_binario
X_teste = X_teste_binario
y_teste = y_teste_binario

# Configurar os parâmetros da Rede Neural para classificação binária
configuracao_binaria = {
    "input_size": X_treino.shape[1],  # Número de características (features) de entrada
    "hidden_size": 10,  # Camada oculta com 10 neurônios para capturar padrões complexos
    "output_size": 1,  # Uma saída para prever 0 ou 1
    "activation": "relu",  # Função de ativação para introduzir não-linearidade
    "loss": "mse",  # Função de perda para minimizar erros quadráticos médios
    "learning_rate": 0.01,  # Taxa de aprendizado para ajustes dos pesos
    "epochs": 1000  # Iterações completas sobre o conjunto de treino
}

# Instanciar e treinar a Rede Neural
rede_neural_binaria = RedeNeural(
    entrada_tamanho=configuracao_binaria["input_size"],
    oculta_tamanho=configuracao_binaria["hidden_size"],
    saida_tamanho=configuracao_binaria["output_size"],
    ativacao=configuracao_binaria["activation"],
    perda=configuracao_binaria["loss"]
)

# Treinar a Rede Neural
rede_neural_binaria.treinar(
    X_treino.to_numpy(),
    y_treino.to_numpy().reshape(-1, 1),
    configuracao_binaria["epochs"],
    configuracao_binaria["learning_rate"]
)

# Avaliar o modelo no conjunto de teste
y_predito_binario = rede_neural_binaria.propagacao_frente(X_teste.to_numpy())
acuracia_binaria = (y_predito_binario.round() == y_teste.to_numpy().reshape(-1, 1)).mean() * 100
print(f"Acurácia do modelo (Classificação Binária): {acuracia_binaria:.2f}%")


Época 0/1000, Perda: 0.2501
Época 100/1000, Perda: 0.2415
Época 200/1000, Perda: 0.2363
Época 300/1000, Perda: 0.2332
Época 400/1000, Perda: 0.2312
Época 500/1000, Perda: 0.2300
Época 600/1000, Perda: 0.2292
Época 700/1000, Perda: 0.2286
Época 800/1000, Perda: 0.2281
Época 900/1000, Perda: 0.2275
Acurácia do modelo (Classificação Binária): 64.42%


Treinamento para Classificação Multiclasse

In [11]:
# Configuração para o treino de classificação multiclasse
# Este bloco implementa o treinamento para prever classes múltiplas.
# Exemplos: categorias como níveis de obesidade ou diagnósticos diferenciados.

# Selecionar o conjunto de dados para classificação multiclasse
X_treino = X_treino_multiclasse
y_treino = y_treino_multiclasse
X_teste = X_teste_multiclasse
y_teste = y_teste_multiclasse

# Configurar os parâmetros da Rede Neural para classificação multiclasse
configuracao_multiclasse = {
    "input_size": X_treino.shape[1],
    "hidden_size": 15,  # Camada oculta mais robusta para lidar com mais classes
    "output_size": len(y_treino.unique()),  # Número de classes diferentes
    "activation": "relu",
    "loss": "cross_entropy",  # Função de perda para tarefas multiclasse
    "learning_rate": 0.01,
    "epochs": 1200  # Mais iterações devido à complexidade da tarefa
}

# Instanciar e treinar a Rede Neural
rede_neural_multiclasse = RedeNeural(
    entrada_tamanho=configuracao_multiclasse["input_size"],
    oculta_tamanho=configuracao_multiclasse["hidden_size"],
    saida_tamanho=configuracao_multiclasse["output_size"],
    ativacao=configuracao_multiclasse["activation"],
    perda=configuracao_multiclasse["loss"]
)

# Treinar a Rede Neural
rede_neural_multiclasse.treinar(
    X_treino.to_numpy(),
    pd.get_dummies(y_treino).to_numpy(),  # Codificar rótulos como one-hot encoding
    configuracao_multiclasse["epochs"],
    configuracao_multiclasse["learning_rate"]
)

# Avaliar o modelo no conjunto de teste
y_predito_multiclasse = rede_neural_multiclasse.propagacao_frente(X_teste.to_numpy())
acuracia_multiclasse = (y_predito_multiclasse.argmax(axis=1) == y_teste.to_numpy()).mean() * 100
print(f"Acurácia do modelo (Classificação Multiclasse): {acuracia_multiclasse:.2f}%")


Época 0/1200, Perda: 0.6929
Época 100/1200, Perda: 0.4414
Época 200/1200, Perda: 0.4122
Época 300/1200, Perda: 0.4120
Época 400/1200, Perda: 0.4118
Época 500/1200, Perda: 0.4116
Época 600/1200, Perda: 0.4114
Época 700/1200, Perda: 0.4113
Época 800/1200, Perda: 0.4111
Época 900/1200, Perda: 0.4109
Época 1000/1200, Perda: 0.4107
Época 1100/1200, Perda: 0.4105
Acurácia do modelo (Classificação Multiclasse): 0.00%


Treinamento para Regressão

In [12]:
# Configuração para o treino de regressão
# Este bloco implementa o treinamento para prever valores contínuos.
# Exemplos: preço de imóveis, medições físicas ou previsões de métricas.

# Selecionar o conjunto de dados para regressão
X_treino = X_treino_regressao
y_treino = y_treino_regressao
X_teste = X_teste_regressao
y_teste = y_teste_regressao

# Configurar os parâmetros da Rede Neural para regressão
configuracao_regressao = {
    "input_size": X_treino.shape[1],
    "hidden_size": 8,  # Camada oculta mais simples devido à natureza contínua da saída
    "output_size": 1,  # Uma saída contínua
    "activation": "tanh",  # Função de ativação para capturar padrões complexos
    "loss": "mse",  # Função de perda para tarefas de regressão
    "learning_rate": 0.01,
    "epochs": 800  # Menos iterações devido à simplicidade do problema
}

# Instanciar e treinar a Rede Neural
rede_neural_regressao = RedeNeural(
    entrada_tamanho=configuracao_regressao["input_size"],
    oculta_tamanho=configuracao_regressao["hidden_size"],
    saida_tamanho=configuracao_regressao["output_size"],
    ativacao=configuracao_regressao["activation"],
    perda=configuracao_regressao["loss"]
)

# Treinar a Rede Neural
rede_neural_regressao.treinar(
    X_treino.to_numpy(),
    y_treino.to_numpy().reshape(-1, 1),
    configuracao_regressao["epochs"],
    configuracao_regressao["learning_rate"]
)

# Avaliar o modelo no conjunto de teste
y_predito_regressao = rede_neural_regressao.propagacao_frente(X_teste.to_numpy())
erro_medio = ((y_predito_regressao - y_teste.to_numpy().reshape(-1, 1))**2).mean()
print(f"Erro Médio Quadrático (Regressão): {erro_medio:.4f}")


Época 0/800, Perda: 0.0569
Época 100/800, Perda: 0.0378
Época 200/800, Perda: 0.0376
Época 300/800, Perda: 0.0376
Época 400/800, Perda: 0.0376
Época 500/800, Perda: 0.0376
Época 600/800, Perda: 0.0376
Época 700/800, Perda: 0.0376
Erro Médio Quadrático (Regressão): 0.0360
